# Product mathiching based on ML algorithms

The project is done by Basang Basangov.
Telegram: [basan4ik](https://t.me/basan4ik)

## 1. Project Description

Data matching is the workflow process of comparing different data values in structured or unstructured format based on similarity or an underlying entity [[1]](https://www.width.ai/post/data-matching-software#:~:text=How%20you%20can%20use%20machine,similarity%20or%20an%20underlying%20entity.). This notebook provides a workflow of how this type of task can be solved using two-staged process: first, FAISS similarity search is used to get 100-200 the most similar products from a database (that could petentially consist of billions of items), then among those 100-200 products get 5 that are even more similar using supervised ML algoritm, which in our case would be LightGBM classification algorithm. The final evaluation metric is accuracy@5.

There are 4 files:
- 'base.csv' is a dataset of anonymized set of products. Each product is presented with a unique id (0-base, 1-base, etc.) and vectors of the shape 1 x 72;
- train.csv is a training dataset. Each row has an id (0-query, 1-query, etc.), a vector (1x72), and id from 'base';
- validation.csv is a dataset of vectors that we need to find the most similar vectors from 'base'.
- validation_answer.csv is a dataset with the right answers to the previous dataset.

This is a project from [Yandex Practicum Masterskaya](https://practicum.yandex.ru/masterskaya/).

## 2. Data Preparation

### 2.1 Installations

In [248]:
# Uncomment if needed
#!pip install faiss-gpu
#!pip install catboost

In [249]:
import pandas as pd
import numpy as np
import faiss

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb

### 2.2 Data Loading

In [250]:
base = pd.read_csv('data/base.csv', index_col=0)
train = pd.read_csv('data/train.csv', index_col=0)

### 2.3 Feature Preparation

In [251]:
scaler = StandardScaler()
scaler.fit(base)

base_scaled = scaler.transform(base)

In [252]:
base_scaled_df = pd.DataFrame(base_scaled, index=base.index)

In [253]:
train_target = train['Target']
train_features = train.drop(['Target'], axis=1)

train_features_scaled = scaler.transform(train_features)

## 3. FAISS Similarity Search

### 3.1 FAISS Index

In [254]:
d = 72                           # dimensions
k_ann = 100                      # k appr. nearest neighbors
nb = 2918139                      # database size
nq = 10000                       # queries. intentionally decreased because of ram capacity
np.random.seed(1234)             # make reproducible

In [255]:
try:
    res = faiss.StandardGpuResources() # use a single GPU
    # make it a flat GPU index
    index_flat = faiss.IndexFlatL2(d)   # build the index

    # make it a flat GPU index
    gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

    # Then we train the index to find a suitable clustering
    gpu_index_flat.train(np.ascontiguousarray(base_scaled.astype('float32')))

    # Finally we add all embeddings to the index
    gpu_index_flat.add(np.ascontiguousarray(base_scaled.astype('float32')))

    print(f'ntotal: {gpu_index_flat.ntotal}')

except:
    print('No GPU found. Using CPU...')
    index = faiss.IndexFlatL2(d)

    # add vectors to the index
    index.add(np.ascontiguousarray(base_scaled.astype('float32'))) 

    print(f'ntotal: {index.ntotal}')

No GPU found. Using CPU...
ntotal: 2918139


### 3.2 FAISS Search

In [256]:
try:
    # download prepared Faiss query indicies of 100 neighbors on full train
    I = np.load('data/i_ndarray.npy')

    # decrease query indicies as there is not enough RAM on my laptop
    if len(I) > 10000:
        I = I[:10000]
except FileNotFoundError:
    print('No prepared files are found.')
    try:
        res = faiss.StandardGpuResources()
        # searching train vectors
        D, I = gpu_index_flat.search(np.ascontiguousarray(train_features_scaled[:nq, :].astype('float32')), k_ann)
    except:
        D, I = index.search(np.ascontiguousarray(train_features_scaled[:nq, :].astype('float32')), k_ann)


In [257]:
I.shape

(10000, 100)

In [258]:
base_index = {k: v for k, v in enumerate(base.index.to_list())}

acc = 0
for target_base_name, k_closest_vectors in zip(train_target[:1000].values.tolist(), I.tolist()):
    acc += int(target_base_name in [base_index[v] for v in k_closest_vectors])

print(100 * acc / len(I))

7.86


In [259]:
len(I)

10000

## 4. LightGBM Classfication

### 4.1 Preparing Features for a Classifier

In [260]:
def build_hstaked_matrix(matrix1, matrix2, index, base_df, y=None, k_ann=100, dims=72, qn=10000):
    '''
    Takes two scaled 3D (qn x k_ann x dims) matricies 
    and transforms them into horizontally stacked 2D matrix;
    along with y vector if it's present

    matrix1 is train or validation matrix;
    matrix2 is a base index-filtered matrix
    '''
    # prepare matrix1 - tile rows
    matrix1_tiled = np.tile(matrix1[:qn], k_ann).reshape(qn, k_ann, dims)
    
    # prepare matrix2
    index_flat = index.ravel()  # flatten index
    # take only those from base that are in flatten
    index_base = matrix2[index_flat].reshape(qn, k_ann, dims)  # create new 3d array
    
    if y is not None:
        last_dim = 1 # it needs for reshape in final result
        if np.all(y == 1):
            last_axis = y
        else:
            y_tiled = np.tile(y[:qn], k_ann)
            I_base_names = base_df.index[index_flat]
            last_axis = np.equal(y_tiled, I_base_names).astype('int').reshape(qn, k_ann, 1)

        # Concatenate the two arrays on axis=2 along with the check_equal array 
        result = np.concatenate([matrix1_tiled, index_base, last_axis], axis=2)
    else:
        last_dim = 0
        result = np.concatenate([matrix1_tiled, index_base], axis=2)

    result = result.reshape(qn*k_ann, (dims*2)+last_dim)

    return result

In [261]:
train_final = build_hstaked_matrix(train_features_scaled, base_scaled, I, base, train_target)

In [262]:
np.average(train_final[..., -1])*100

0.00019999999999999998

### 4.2 Fighting class imbalance

To balance target classes let's augment our train matrix

In [263]:
train_tail_90000 = train_features_scaled[10_000:, :]
tail_base_90000 = base_scaled_df.loc[train.values[10000:, -1]]

train_final_tail_90000 = np.concatenate([train_tail_90000, tail_base_90000.values, np.ones((90000, 1))], axis=1)

train_final_tail_90000.shape

(90000, 145)

In [264]:
train_final = np.concatenate([train_final, train_final_tail_90000], axis=0)

In [265]:
X = train_final[:, :-1]
y = train_final[:, -1]

In [266]:
np.average(y)

0.08257064220183487

In [267]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=42,
                                                    stratify=y)

### 4.3 Model Training

In [268]:
# baseline LightGBM model
lgbm_clf = lgb.LGBMClassifier(n_jobs=-1, random_state=42)

lgbm_clf.fit(X_train, y_train)

LGBMClassifier(random_state=42)

In [269]:
print('LightGBM Classifier accuracy score:', lgbm_clf.score(X_test, y_test))

LightGBM Classifier accuracy score: 0.9445412844036697


## Validation set

In [270]:
validation = pd.read_csv('data/validation.csv', index_col=0)
validation_answer = pd.read_csv('data/validation_answer.csv', index_col=0)

In [271]:
validation_scaled = scaler.transform(validation)

In [272]:
k_ann=100
try:
    res = faiss.StandardGpuResources()
    # searching validation vectors
    D, I = gpu_index_flat.search(np.ascontiguousarray(validation_scaled[:nq, :].astype('float32')), k_ann)
except:
    D, I = index.search(np.ascontiguousarray(validation_scaled[:nq, :].astype('float32')), k_ann)


In [273]:
# by default it takes only first 10_000 rows from validation set
validation_final = build_hstaked_matrix(validation_scaled, base_scaled, I, base)

In [274]:
qn = 10_000
k_ann = 100
y_tiled = np.tile(validation_answer.iloc[:qn].values, k_ann).reshape(qn*k_ann,)
I_base_names = base.index[I.ravel()]
last_axis = np.equal(y_tiled, I_base_names).astype('int')

In [275]:
print('LightGBM Classifier accuracy score on validation set:', lgbm_clf.score(validation_final, last_axis))

LightGBM Classifier accuracy score on validation set: 0.993443


In [276]:
valid_answer_10000 = validation_answer['Expected'].iloc[:qn].values

In [277]:
proba = lgbm_clf.predict_proba(validation_final)

In [278]:
valid_prob_index = list(zip(proba[:, -1], 
                            base.index[I.ravel()].values))  # 1mln of (0.023446027463881292, '3209652-base')-like tuples

In [279]:
# accuracy@5 metric
acc = 0
chunk_size = 100
for i in range(10000):
    start_index = i * chunk_size
    end_index = start_index + chunk_size

    chunk = valid_prob_index[start_index:end_index]
    top_5 = sorted(chunk, reverse=True)[:5]
    if valid_answer_10000[i] in [t[1] for t in top_5]:
        acc+=1

print(100 * acc / len(I))

62.69


accuracy@5 score on validation[:10000] is 62.69

## Conclusion

Due to restrictions of RAM capacity on my laptop, the decision was made to use only first 10_000 rows of train and validation datasets. Nevertheless, two-staged similarity search (FAISS + LightGBM classifier) showed that the approach is working. 

There are some quirks that could be changed:
- I used FAISS IndexFlatL2 (Exact Search for L2), as it doesn't take a lot of time, but theoretically should provide the best results in terms of search quality.
- We could use another number of approximate nearest neighbors, I used 100.
- Using cloud solutions with larger RAM, we could perform training on a full dataset.
- I used LightGBM classifier out of the box, other ML algorithms can be tested as well as trying various hyperparameters.
- last but not least, the code is ugly and probably inefficient, change as you wish.